# Step 4-1. LLM 기반 검색 결과 요약 (RAG 파이프라인)
OpenSearch 벡터 검색 결과를 Claude LLM으로 요약합니다.

## RAG 파이프라인 흐름
1. 사용자 질문을 **Titan 임베딩 모델**로 벡터 변환
2. OpenSearch에서 **k-NN 벡터 검색**으로 유사 문서 5개 검색
3. 검색된 문서들을 **프롬프트에 포함**시켜 Claude LLM에 전달
4. LLM이 검색 결과를 기반으로 **한국어 요약** 생성

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0

## 1. 설정 (Configuration)

In [9]:
import os, json

# Step 0에서 저장한 설정 불러오기
try:
    with open("../config.json") as f:
        _config = json.load(f)
    print("✅ config.json 로드 완료")
except FileNotFoundError:
    raise FileNotFoundError("❌ config.json을 찾을 수 없습니다. Step 0 노트북을 먼저 실행해주세요.")

HOST = _config.get("OPENSEARCH_HOST")
if not HOST:
    raise ValueError("❌ config.json에 OPENSEARCH_HOST 값이 없습니다. Step 0 노트북을 먼저 실행해주세요.")
DEFAULT_REGION = _config.get("DEFAULT_REGION", "ap-northeast-2")
BEDROCK_REGION = _config.get("BEDROCK_REGION", "us-east-1")
PROFILE = _config.get("PROFILE", "skku-opensearch-session")

index_name = 'bedrock-test'
embedding_model_id = 'amazon.titan-embed-text-v2:0'
# llm_model_id = 'us.anthropic.claude-opus-4-6-v1'
llm_model_id = 'us.anthropic.claude-sonnet-4-5-20250929-v1:0'
# TODO: 검색할 질문을 바꿔보세요!
user_query = "s3와 cloudfront"

✅ config.json 로드 완료


## 2. OpenSearch 클라이언트 생성

In [10]:
import boto3
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection

service = 'aoss'
credentials = boto3.Session(profile_name=PROFILE).get_credentials()
auth = AWSV4SignerAuth(credentials, DEFAULT_REGION, service)

client = OpenSearch(
    hosts=[{'host': HOST, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

print("OpenSearch 클라이언트 생성 완료")

OpenSearch 클라이언트 생성 완료


## 3. Bedrock 클라이언트 생성 및 임베딩 함수 정의

In [11]:
import json

print(f"Creating a boto3 session with profile '{PROFILE}'...")
session = boto3.Session(profile_name=PROFILE)

print(f"Creating a Bedrock client in region: {BEDROCK_REGION}")
bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name=BEDROCK_REGION,
)
print("Bedrock client created successfully.")

def get_embedding_from_bedrock(text, model_id):
    """Bedrock API를 호출하여 주어진 텍스트의 벡터 임베딩을 반환합니다."""
    body = json.dumps({"inputText": text})
    response = bedrock_client.invoke_model(
        body=body,
        modelId=model_id,
        accept="application/json",
        contentType="application/json",
    )
    response_body = json.loads(response.get("body").read())
    return response_body.get("embedding")

Creating a boto3 session with profile 'skku-opensearch-session'...
Creating a Bedrock client in region: us-east-1
Bedrock client created successfully.


In [12]:
# 인덱스에 데이터가 업로드되었는지 확인
if not client.indices.exists(index=index_name):
    print(f"❌ '{index_name}' 인덱스가 존재하지 않습니다.")
    print("   → Step 3-0 (Titan 임베딩 데이터 업로드) 노트북을 먼저 실행해주세요.")
else:
    _doc_count = client.count(index=index_name)['count']
    if _doc_count == 0:
        print(f"⏳ '{index_name}' 인덱스는 있지만 검색 가능한 문서가 0개입니다.")
        print("   → 데이터 업로드 직후라면 인덱싱 중일 수 있습니다. 잠시 후 다시 실행해주세요.")
    else:
        print(f"✅ '{index_name}' 인덱스에 {_doc_count}개의 문서가 준비되어 있습니다.")

✅ 'bedrock-test' 인덱스에 50개의 문서가 준비되어 있습니다.


## 4. 벡터 검색 수행

In [13]:
print(f"\n1. Generating embedding for the user query: '{user_query}'")
query_vector = get_embedding_from_bedrock(user_query, embedding_model_id)
print("   - Embedding generated successfully.")

print("\n2. Performing k-NN search on OpenSearch...")
knn_query = {
    "size": 5,
    "query": {
        "knn": {
            "content_vector": {
                "vector": query_vector,
                "k": 5
            }
        }
    },
    "_source": ["title", "content", "author", "post_id"]
}

response = client.search(index=index_name, body=knn_query)
print("   - Search completed successfully.")

hits = response["hits"]["hits"]
print(f"\n3. Found {len(hits)} relevant documents:")
for i, hit in enumerate(hits):
    print(f"   - Doc {i+1}: {hit['_source']['title']} (Score: {hit['_score']})")


1. Generating embedding for the user query: 's3와 cloudfront'
   - Embedding generated successfully.

2. Performing k-NN search on OpenSearch...
   - Search completed successfully.

3. Found 5 relevant documents:
   - Doc 1: Terraform으로 개인 블로그 인프라 구축 후기 (Score: 1.254173)
   - Doc 2: WebRTC를 이용한 실시간 화상 채팅 웹앱 (Score: 1.2202702)
   - Doc 3: Docker와 Kubernetes의 차이점이 뭔가요? (Score: 1.1915131)
   - Doc 4: Three.js로 만든 3D 인터랙티브 웹사이트 (Score: 1.187766)
   - Doc 5: 웹소켓(WebSocket)과 HTTP의 차이점 (Score: 1.1756728)


## 5. LLM으로 검색 결과 요약

In [14]:
# 요약할 내용을 하나의 문자열로 합치기
context_for_llm = ""
for hit in hits:
    context_for_llm += f"문서 제목: {hit['_source']['title']}\n"
    context_for_llm += f"문서 내용: {hit['_source']['content']}\n\n"

prompt = f"""
Human: 다음은 OpenSearch 검색 결과입니다. 이 내용을 바탕으로 사용자의 질문에 대해 한국어로 친절하게 요약해 주세요. 대신 질문과 관련 없는 결과는 제외하고 요약해주세요. 그리고 제외된 결과는 마지막에 글 제목 리스트를 알려주세요.

[검색 결과]
{context_for_llm}

[질문]
{user_query}

Assistant:
"""

print("\n4. Summarizing the search results using Bedrock LLM...")

body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    ]
})

response = bedrock_client.invoke_model(
    body=body,
    modelId=llm_model_id,
    accept="application/json",
    contentType="application/json"
)
response_body = json.loads(response.get('body').read())
summary = response_body['content'][0]['text']
print("   - Summarization completed successfully.")

print("\n" + "="*50)
print(" [최종 요약 결과] ")
print("="*50)
print(summary)
print("="*50)


4. Summarizing the search results using Bedrock LLM...
   - Summarization completed successfully.

 [최종 요약 결과] 
# 검색 결과 요약

사용자님의 질문 "s3와 cloudfront"와 관련된 내용을 요약해드리겠습니다.

## 📌 관련 내용

**Terraform으로 개인 블로그 인프라 구축 사례**

- AWS의 **S3**와 **CloudFront**, 그리고 Route 53을 활용하여 정적 웹사이트를 배포한 사례입니다.
- **Terraform**을 사용하여 IaC(Infrastructure as Code) 방식으로 인프라를 코드로 관리했습니다.
- 이러한 구성을 통해 인프라 관리가 매우 편리해졌다고 합니다.
- 전체 구현 코드는 깃허브에 공유되어 있습니다.

💡 **핵심 포인트**: S3와 CloudFront는 정적 웹사이트 호스팅에 자주 사용되는 AWS 서비스 조합으로, S3는 파일 저장소 역할을, CloudFront는 CDN(콘텐츠 전송 네트워크) 역할을 담당합니다.

---

### 제외된 검색 결과
다음 문서들은 질문과 직접적인 관련이 없어 제외되었습니다:
- WebRTC를 이용한 실시간 화상 채팅 웹앱
- Docker와 Kubernetes의 차이점이 뭔가요?
- Three.js로 만든 3D 인터랙티브 웹사이트
- 웹소켓(WebSocket)과 HTTP의 차이점


## 실습 과제

### 과제: 다양한 LLM으로 테스트하고 결과와 응답 속도를 비교해보기
위 설정 셀에서 `llm_model_id`를 변경하여 다시 실행해보세요.
- `us.anthropic.claude-sonnet-4-5-20250929-v1:0` (기본)
- `us.anthropic.claude-opus-4-6-v1`

`user_query`도 다양하게 바꿔보며 요약 품질을 비교해보세요.